In [1]:
from gensim.models import word2vec
import gensim.models
from sklearn.decomposition import PCA
import matplotlib
import matplotlib.pyplot as plt
from numba import jit# 加速套件
import glob
import os
import statistics as stat
from scipy import stats
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import xlsxwriter
import numpy as np
import numpy as ndarray
from gensim.models import word2vec
#from gensim.models import Word2vec
from gensim import models
import logging
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
import jieba
from gensim.test.utils import datapath

# 以下是順序

In [2]:
"""
1.載入檔案,這個檔案是每篇文章的內文與平均向量,拿出平均向量那欄,存到articles_matrix裡面
2.載入bin檔,這個檔案是訓練好的bin檔,裡面有每個詞的向量
測試開始
3.輸入文字:def 請輸入文字(input_words)把輸入的文章斷詞成字
4.獲取輸入詞的平均向量:def get_article_avgvector(wordlist):把斷完詞的單字利用先前載入的bin檔依序變成向量,加總後再取平均
於是現在有了輸入的文章平均詞向量,接著,只要與原本的文章向量做比對,就可以找到最相近的文章
5.餘弦相似度:def cos_sim(vector_a, vector_b):這個函數把輸入的向量跟5000篇新聞逐一比對餘弦相似度
6.比對:def 餘閒相似度找文章(input_words):這個函數把5.比對的結果存起來,再找出最相近的一篇列出來
"""

'\n1.載入檔案,這個檔案是每篇文章的內文與平均向量,拿出平均向量那欄,存到articles_matrix裡面\n2.載入bin檔,這個檔案是訓練好的bin檔,裡面有每個詞的向量\n測試開始\n3.輸入文字:def 請輸入文字(input_words)把輸入的文章斷詞成字\n4.獲取輸入詞的平均向量:def get_article_avgvector(wordlist):把斷完詞的單字利用先前載入的bin檔依序變成向量,加總後再取平均\n於是現在有了輸入的文章平均詞向量,接著,只要與原本的文章向量做比對,就可以找到最相近的文章\n5.餘弦相似度:def cos_sim(vector_a, vector_b):這個函數把輸入的向量跟5000篇新聞逐一比對餘弦相似度\n6.比對:def 餘閒相似度找文章(input_words):這個函數把5.比對的結果存起來,再找出最相近的一篇列出來\n'

In [121]:
#1.載入文章檔案
articles = pd.read_excel(r"C:\Users\Big data\Desktop\class\funcardproject\LDA\LDA_向量_20群.xlsx")
articles

5594

In [55]:
#1.載入LDA分群檔案
group_lists = pd.read_excel(r"C:\Users\Big data\Desktop\class\funcardproject\LDA\LDA_20群_100詞.xlsx")
group_lists

,Unnamed: 0,label,word,freq
0,0,0,"['我們', '你', '我', '被', '就是', '這個', '民眾', '可以', ...",0
1,1,1,"['大陸', '我', '你', '就是', '日本', '現在', '可以', '因為',...",0
2,2,2,"['新聞', '活動', '您', '金幣', '該', '好', '按', '優質', '...",0
3,3,3,"['贊助', '與', '台灣', '心中', '中獎', '服務', '金融', '網點'...",0
4,4,4,"['日', '餐廳', '飯店', '美食', '台北', '吃', '送', '及', '...",0
5,5,5,"['旅遊', '台灣', '日', '旅客', '日本', '台北', '機場', '元',...",0
6,6,6,"['銀行', '金管會', '資料', '表示', '客戶', '業務', '交易', '或...",0
7,7,7,"['回饋', '消費', '優惠', '與', '元', '現金回饋', '最高', '卡'...",0
8,8,8,"['繳稅', '蘋果', 'IPHONE', '繳納', '報稅', '申報', '遊戲',...",0
9,9,9,"['他', '總統', '我', '他們', '被', '年', '立委', '人', '美...",0


In [32]:
#2.載入bin檔
wv_from_bin = KeyedVectors.load_word2vec_format(datapath(r'C:\Users\Big data\Desktop\class\funcardproject\word2vec_model\news\測試100win20cbow1.bin'), binary=True) 

# 測試區

In [140]:
input_words= input()
input_vector_matrix =請輸入文字(input_words)
label= get_label(input_words)
group_articles = group_article(label)
articles_matrix =[get_article_matrix(group_articles,i) for i in range(len(group_articles))]
餘閒相似度找文章(input_words)

滙豐銀推升級現金回饋御璽卡 哩程數1元換2哩被號稱是刷卡神卡的中國信託LINE Pay卡，今年度回饋縮減後，各家信用卡後來居上爭搶刷卡神卡地位。外商銀行滙豐銀行力推自家現金回饋御璽卡，2019年宣布全面升級，除了原先不設門檻國內消費1.22％、海外消費2.22％現金回饋外，更首次開放卡友可以將回饋金累計兌換飛行里程數。此次全面升級回饋金累


ValueError: DataFrame constructor not properly called!

In [53]:
#測試開始_餘閒相似度
input_words= input()
input_vector_matrix =請輸入文字(input_words)
餘閒相似度找文章(input_words)

玉山
['玉山']

這幾個字的平均向量是:
[[ 0.19610208 -0.563665   -0.3372065  -0.5663267   0.10419251 -0.03174496
  -0.57335347 -0.01868376 -0.77252775 -0.3306109   0.44495112  0.50069845
  -0.19555637  0.08804778 -0.7298876  -0.24601893 -0.04557158 -0.38264757
  -0.18670572 -0.48187992  0.4048067  -0.17634086  0.07316504 -0.4861287
  -0.5141417   0.6257923   0.61300194  0.27603537 -0.15095751  0.04430315
   0.12776838 -0.11363579 -0.1937178  -0.04849094 -1.1038033  -0.16659811
   0.3701372   0.95391536 -0.35162738 -0.6978629   0.1354421   0.17245938
  -0.27713624 -0.29105332  0.42139494 -0.44791517 -0.43316543 -0.51872057
  -0.32615614  0.17171097  0.01827458 -0.25880343  0.50627005 -0.06921496
   0.5086911  -0.49112234 -0.09658755  0.01427571 -0.7565906  -0.39970285
   0.02148487 -0.17707984 -0.90400654  0.42158422 -0.02072658  0.9841369
   0.6100574  -0.5122336  -0.73656815 -0.5097141  -0.16665676  0.1635552
  -0.22809388  0.25000995 -0.2141154  -0.8958415  -0.07857244  0.4800984
   0.4884535  -0.41

# 根據分群選項輛

In [96]:
def group_article(label):
    group_articles=pd.DataFrame(columns=['Unnamed: 0','No.','content','article_vector_matrix','label'])
    cc=0
    for n,i in enumerate(articles['label']):
        if i == label:
            group_articles.loc[cc]=list(articles.loc[n])
            cc+=1
    return group_articles

In [35]:
def get_label(input_words):
    group_lists["freq"]=0
    for n,group in enumerate(group_lists["word"]):
        for input_word in input_words:
            if input_word in group:
                group_lists["freq"][n]+=1
    label = np.where(group_lists["freq"]==np.max(group_lists["freq"]))
    label=label[0][0]
    return label

In [36]:
#4.獲取輸入詞的平均向量
def get_article_avgvector(wordlist):
    #取每篇文章平均向量
    # x=np.matrix(wv_from_bin[word])安安?
    len_wordlist=0
    input_avgvector_matrix=0
    for word in wordlist:

        try:
            x=np.matrix(wv_from_bin[word])
            input_avgvector_matrix+=x
            len_wordlist+=1

        except:
            pass
    if type(input_avgvector_matrix)==int:
        input_avgvector_matrix = articles_matrix[1]
    else:
        input_avgvector_matrix = input_avgvector_matrix/len_wordlist
    
 
      
    #若沒有輸入文章或輸入的詞不在bin檔則回傳第0篇文章向量

    
     
    return(input_avgvector_matrix)

In [37]:
x=np.matrix(wv_from_bin["旅遊"])
y=np.matrix(wv_from_bin["加油"])
input_avgvector_matrix=0
input_avgvector_matrix+=x
input_avgvector_matrix+=y
input_avgvector_matrix

matrix([[ 0.49321005, -0.97627735, -1.4002988 , -0.68763477, -0.1221542 ,
         -0.5081506 , -1.8594931 , -0.61623746,  0.24533285, -0.50148046,
         -0.57171524,  0.11776888, -0.07833362, -1.0480946 , -0.02709624,
         -0.6206844 , -0.09482106,  0.6255164 , -0.9778987 , -0.76802504,
         -0.08358138, -0.09517649,  1.5977589 ,  0.35336128, -0.48482433,
          0.0540897 ,  0.91026616, -0.3784378 ,  1.6129799 , -0.3451628 ,
         -1.5321958 ,  0.13388553,  0.05983865, -1.3810012 , -0.81214154,
          0.19814089, -0.17143549,  0.8975501 , -0.08294594, -0.7686179 ,
         -0.84542733,  0.02836706, -0.22351918, -0.0919289 ,  0.6135563 ,
          1.719913  , -1.4062992 ,  0.61917096,  0.41225258,  0.3907897 ,
          0.16261263,  0.84341156, -0.21983007, -0.19169547, -0.39086354,
         -1.2234123 , -0.7592441 ,  0.10992469, -0.70064735, -0.8791137 ,
         -1.1320727 , -0.4950929 ,  0.19753097, -0.47997445,  0.27035797,
          0.5783057 ,  0.5555821 ,  0.

In [38]:
#3.輸入文字
def 請輸入文字(input_words):
    #隨意輸入文字
    #轉DataFrame
    # input_words=pd.DataFrame(input_words,columns=['content'])
    #斷詞
    wordlist=jieba.lcut(input_words,cut_all=False)
    # df1=斷詞(input_words)
    #清乾淨
    # news = 整理(cut)
    # 吐出來
    # output_words = input_words['seg'][0]
    # output_words = output_words.split(' ')
    print(wordlist)
    input_vector_matrix = get_article_avgvector(wordlist)
    print()
    print("這幾個字的平均向量是:")
    print(input_vector_matrix)
    return(input_vector_matrix)
    return(wordlist)

In [139]:
#3.輸入文字
def 請輸入文字(input_words):
    #隨意輸入文字
    #轉DataFrame
    input_words=pd.DataFrame(input_words,columns=['content'])
    #斷詞
#     wordlist=jieba.lcut(input_words,cut_all=False)
    df1=斷詞(input_words)
    #清乾淨
    input_words = 整理(cut)
    # 吐出來
    output_words = input_words['seg'][0]
    output_words = output_words.split(' ')
    print(output_words)
    input_vector_matrix = get_article_avgvector(output_words)
    print()
    print("這幾個字的平均向量是:")
    print(input_vector_matrix)
    return(input_vector_matrix)
    return(wordlist)

In [39]:
#5.餘弦相似度
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a 
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    return sim

In [40]:
#撈出存在excel的向量轉為matrix
def get_article_matrix(article,i):
    aa = article.loc[:,["article_vector_matrix"]][i:i+1]
    #轉ARRAY再轉list
    b = np.array(aa)
    b=b[0].tolist()#list
    # 切
    c = str(b[0]).split(',')
    article_matrix = np.mat(c).astype(float)
    return(article_matrix)

In [110]:
articles_matrix_list=[]
for b in range(len(group_articles)):
    結果=cos_sim(input_vector_matrix,articles_matrix[b])
    articles_matrix_list.append(結果)
#     print("第",articles_matrix_list.index(max(articles_matrix_list)),"篇新聞最相似")
print("第",articles_matrix_list.index(max(articles_matrix_list)),"篇新聞最相似")
哭喔 = articles_matrix_list.index(max(articles_matrix_list))
print("文章內文為:","\n","\n","------------------------------------------------------------","\n",np.array(group_articles[哭喔:哭喔+1]['content'])[0])


第 18 篇新聞最相似
文章內文為: 
 
 ------------------------------------------------------------ 
 〈獨家〉自助加油試身「手」　每公升2元省很大
自助加油降幅相對大了些，但受不同信用卡影響，折扣也有高低差別，至於人工加油，50公升油箱汽車用信用卡5%現金回饋，搭配指定加油站，每公升降1.7元優惠，也能省下破百元，但降幅還是沒有自助加油來的低，開車族想省荷包，還是得先習慣自己加油。 


In [122]:
#比對
def 餘閒相似度找文章(input_words):
    articles_matrix_list=[]
    for b in range(len(group_articles)):
        結果=cos_sim(input_vector_matrix,articles_matrix[b])
        articles_matrix_list.append(結果)
#     print("第",articles_matrix_list.index(max(articles_matrix_list)),"篇新聞最相似")
    print("第",articles_matrix_list.index(max(articles_matrix_list)),"篇新聞最相似")
    哭喔 = articles_matrix_list.index(max(articles_matrix_list))
    print("文章內文為:","\n","\n","------------------------------------------------------------","\n",np.array(group_articles[哭喔:哭喔+1]['content'])[0])
    

In [138]:
# 斷詞套件
def 斷詞(news):
    df1=news
    df1['seg']=None
    df1['word_freq']=None
    n=len(news)
    alltext=[]
    stopset=set()
    stop2=['/n','']
    with open('C:\\Users\\Big data\\Desktop\\字詞貼標\\DB104-master\\stop.txt','r',encoding='ISO-8859-1') as s:
        for line in s:
            stopset.add(line.strip('\n'))
    jieba.load_userdict('C:\\Users\\Big data\\Desktop\\字詞貼標\\DB104-master\\userdict.txt')
    for i in range(n):
        seg=''
        wf={}
        text= str(df1.loc[i]['content'])
        cut=jieba.cut(text,cut_all=False)
        for j in cut:
            if j not in stopset:
                seg +=j+' '
                seg = seg.replace("\n",'')
        df1['seg'][i]=seg
        for w in seg.split(' '):
            if w not in wf:
                wf[w]=1
            else:
                wf[w]+=1
        word_list=[(k,wf[k]) for k in wf if k not in stop2 ]
        word_list.sort(key=lambda a :a[1],reverse=True)
        df1['word_freq'][i]=word_list
    return(df1)



# 斷詞整理(去除標點符號)
def 整理(df1):
    datanews = df1["seg"]
    # 轉list
    train_data = np.array(datanews)#np.ndarray()
    datanews=train_data.tolist()#list
    df1=''
    for datanew in datanews:
        df1+=str(datanew)+'\n'
    df1=df1.replace('[','').replace('\'','').replace('「 ','').replace('」 ','').replace('」 ','').replace('、 ','').replace('： ','')
    df1=df1.replace('《 ','').replace('》 ','').replace('） ','').replace('( ','').replace('／ ','').replace('， ','')
    df1=df1.replace('╱ ','').replace('！ ','').replace('？ ','').replace('（ ','').replace('。 ','').replace('； ','').replace('… ','')
    df1=df1.replace(']','').replace(' :','').replace('\u3000','').replace('\n','')
    df2=df1
    for i in range(len(df2)):
        df1=' '.join(df2.split())
    return(df1)